# Finding and Replacing Missing Data

In [2]:
import pandas as pd
import numpy as np
import os 

## Load and Inspect the Data

In [3]:
filename = os.path.join("/Users/salmanyagaka/Documents/interviews/adult.csv")
df = pd.read_csv(filename, header=0)

In [3]:
df.shape

(7000, 15)

In [15]:
df = df.mask(np.random.rand(*df.shape) < 0.05)
df.head()



,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25.0,NaN,226802.0,NaN,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,NaN,40.0,United-States,<=50K
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,NaN
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,NaN,NaN,NaN,White,Male,0.0,NaN,40.0,United-States,NaN
3,44.0,Private,NaN,Some-college,10.0,NaN,NaN,NaN,Black,NaN,7688.0,0.0,NaN,NaN,>50K
4,NaN,?,103497.0,Some-college,10.0,Never-married,?,Own-child,White,NaN,0.0,0.0,30.0,United-States,<=50K



## Dealing with Missing Data

Our goal will be to identify which columns in a dataset have missing values, and to replace a missing value in a column with the mean of the other values in that column. We will add dummy variables to our dataset to indicate which  columns initially had missing values. 

### Step 1:  Identify Missing Values Using Pandas `isnull()` Method

First let us check if there are missing values in DataFrame `df`.

In [20]:
df.isnull().values.any()

np.True_

DataFrame `df` contains missing values! The Pandas `isnull()` method returns `True`/`False` values indicating whether a value is or is not missing in a particular position in a DataFrame or Series. This method recognizes various spellings of missingness like `NaN`, `nan`, `None`, and `NA` among others.<br> Consult the online [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isnull.html) for more information.

In [17]:
df.isnull().head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,False,True,False,True,False,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,True,True,True,False,False,False,True,False,False,True
3,False,False,True,False,False,True,True,True,False,True,False,False,True,True,False
4,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False


The code cell below counts the number of times a missing value occurs in each column. It applies the `isnull()` method and then aggregates the results by columns using the `np.sum()` function. For more information about `np.sum()`, consult the online [documentation](https://numpy.org/doc/stable/reference/generated/numpy.sum.html).

In [18]:
nan_count = np.sum(df.isnull(), axis = 0)
nan_count

age                15001
workclass          14948
fnlwgt             14875
education          15043
educational-num    14952
marital-status     15040
occupation         15169
relationship       15038
race               14962
gender             14865
capital-gain       15247
capital-loss       15053
hours-per-week     14893
native-country     15048
income             14939
dtype: int64

The code cell below stores the names of the columns with detected missing values into a Python list.

In [21]:
condition = nan_count != 0 # look for all columns with missing values

col_names = nan_count[condition].index # get the column names
print(col_names)

nan_cols = list(col_names) # convert column names to list
print(nan_cols)

Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')
['age', 'workclass', 'fnlwgt', 'education', 'educational-num', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']


### Step 2: Choose Which Values to Fill 

We can see that five columns in our DataFrame contain missing values. Would you want to replace the missing values with something for every one of these columns? 

Let's take a look at the data types of the columns that contain missing values using `dtypes`.

In [9]:
nan_col_types = df[nan_cols].dtypes
nan_col_types

age               float64
workclass          object
occupation         object
hours-per-week    float64
native-country     object
dtype: object

For three of the five identified columns, the type is 'object'. Is this a problem?<br>
    A common approach to dealing with the missing values is to replace those values with either the mean, the median, or some other type of 'representative' value wherever a `nan` occurs. This, of course, assumes that the column is numerical to begin with. That does not seem to be true for the `workclass`, `occupation`, and `native-country` variables.
Let us confirm:

In [22]:
print(df['workclass'].unique())
print(df['occupation'].unique())
print(df['native-country'].unique())

[nan 'Private' 'Local-gov' '?' 'Self-emp-not-inc' 'Federal-gov'
 'State-gov' 'Self-emp-inc' 'Without-pay' 'Never-worked']
['Machine-op-inspct' 'Farming-fishing' nan '?' 'Other-service'
 'Craft-repair' 'Adm-clerical' 'Exec-managerial' 'Prof-specialty'
 'Tech-support' 'Sales' 'Priv-house-serv' 'Handlers-cleaners'
 'Transport-moving' 'Protective-serv' 'Armed-Forces']
['United-States' nan '?' 'Peru' 'Mexico' 'Ireland' 'Germany' 'Philippines'
 'Thailand' 'Haiti' 'El-Salvador' 'Vietnam' 'Puerto-Rico'
 'Dominican-Republic' 'Columbia' 'India' 'Poland' 'Laos' 'Cuba' 'England'
 'Taiwan' 'South' 'Italy' 'Canada' 'Portugal' 'China' 'Nicaragua'
 'Honduras' 'Iran' 'Guatemala' 'Scotland' 'Jamaica' 'Yugoslavia' 'Hong'
 'Hungary' 'Ecuador' 'Trinadad&Tobago' 'Japan'
 'Outlying-US(Guam-USVI-etc)' 'Greece' 'Cambodia' 'France'
 'Holand-Netherlands']


The concept of 'mean' is not defined for string entries, so filling in the missing values with the mean of the column wouldn't work here. In real business settings, one way to go about filling in the missing values would be to fit a model that predicts the country based on other values. All data-filling methods come with caveats, and some may threaten the validity of your larger analytical conclusions.

For the rest of this exercise, we will focus only on the numerical variables, for which it makes sense to replace every missing value with the mean of the column. Those are `age` and `hours-per-week` columns.

###  Step 3: Create 'Dummy' Variables for Missing Values

No method of imputing missing values is perfect, and for this reason it makes sense to keep track of which values we artificially created. 

The code cell below looks at the the values in columns `age` and `hours-per-week` and stores the corresponding `True`/`False` values (True if the value is missing and False if the value is present) in new columns `age_na` and `hours-per-week_na`. Run the cell and inspect the new columns.

In [23]:
df['age_na'] = df['age'].isnull()
df['hours-per-week_na'] = df['hours-per-week'].isnull()
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,age_na,hours-per-week_na
0,25.0,NaN,226802.0,NaN,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,NaN,40.0,United-States,<=50K,False,False
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,NaN,False,False
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,NaN,NaN,NaN,White,Male,0.0,NaN,40.0,United-States,NaN,False,False
3,44.0,Private,NaN,Some-college,10.0,NaN,NaN,NaN,Black,NaN,7688.0,0.0,NaN,NaN,>50K,False,True
4,NaN,?,103497.0,Some-college,10.0,Never-married,?,Own-child,White,NaN,0.0,0.0,30.0,United-States,<=50K,True,False


### Step 4: Fill the Missing Values Using Pandas `fillna()` Method

The Pandas `fillna()` method is used to "fill in" missing values in a Series or DataFrame object. Consult the online [documentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.fillna.html) for more information about how to use the `fillna()` method.
The code cell below uses `fillna()` to fill in values for the missing values in the `age` column.
It fills in the missing values with the mean value of all of the existing values in the that column. It uses the Pandas `mean()` method to compute the replacement values. 
For more information about `mean()`, consult the online [documentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.mean.html).

Tip: when working with `fillna()`, make sure that you do not just create a copy object with the filled values, but change the original values of the `df` object by specifying the `inplace = True` parameter value.

First inspect some of the columns that contain missing values.

In [24]:
df.loc[df['age'].isnull()]


,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,age_na,hours-per-week_na
4,NaN,?,103497.0,Some-college,10.0,Never-married,?,Own-child,White,NaN,0.0,0.0,30.0,United-States,<=50K,True,False
5,NaN,Private,NaN,10th,6.0,NaN,Other-service,Not-in-family,NaN,Male,NaN,0.0,30.0,United-States,<=50K,True,False
6,NaN,?,NaN,HS-grad,9.0,Never-married,NaN,Unmarried,Black,Male,0.0,0.0,NaN,NaN,<=50K,True,True
11,NaN,Federal-gov,212465.0,Bachelors,NaN,NaN,Adm-clerical,Husband,NaN,Male,0.0,0.0,NaN,NaN,NaN,True,True
12,NaN,Private,NaN,NaN,9.0,Never-married,Adm-clerical,Not-in-family,NaN,NaN,NaN,NaN,NaN,United-States,<=50K,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48827,NaN,Private,198216.0,Assoc-acdm,12.0,Divorced,Tech-support,Not-in-family,White,Female,NaN,0.0,40.0,United-States,<=50K,True,False
48832,NaN,Private,NaN,10th,6.0,NaN,Handlers-cleaners,Husband,Amer-Indian-Eskimo,Male,NaN,0.0,NaN,NaN,<=50K,True,True
48839,NaN,Private,151910.0,HS-grad,9.0,Widowed,Adm-clerical,Unmarried,White,Female,0.0,NaN,40.0,United-States,<=50K,True,False
48840,NaN,Private,NaN,NaN,9.0,Never-married,Adm-clerical,Own-child,White,NaN,0.0,0.0,20.0,United-States,<=50K,True,False


In [26]:
# look at one row that contains a missing value for age
print("Row 4:  " + str(df['age'][4]))

# compute mean for all non null age values
mean_ages=df['age'].mean()
print("mean value for all age columns: " + str(mean_ages))

# fill all missing values with the mean
df['age'].fillna(value=mean_ages, inplace=True)

# look at one of the rows that contained a missing value for age. 
# It should now contain the mean
print("Row 654:  " + str(df['age'][4]))



Row 4:  38.645075500132975
mean value for all age columns: 38.645075500132975
Row 654:  38.645075500132975


/var/folders/93/rz8r2jvd029bdqgkk56r8z0h0000gn/T/ipykernel_60226/2563807143.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['age'].fillna(value=mean_ages, inplace=True)


In the code cell below, do the same for the `hours-per-week` column.

1. Compute the mean value of the `hours-per-week` column and save the result to variable `mean_hours`
2. Use `fillna` to change the values of the missing columns to `mean_hours`.

### Graded Cell

The cell below will be graded. Remove the line "raise NotImplementedError()" before writing your code.

In [20]:
mean_hours = df['hours-per-week'].mean()

df['hours-per-week'].fillna(value=mean_hours, inplace=True)

print(f"mean hours: {mean_hours}")
print(f"Row 3721 hours: {df['hours-per-week'][3721]}")

mean hours: 40.13001443001443
Row 3721 hours: 40.13001443001443


### Self-Check

Run the cell below to test the correctness of your code above before submitting for grading. Do not add code or delete code in the cell.

In [21]:
# Run this self-test cell to check your code; 
# do not add code or delete code in this cell
from jn import testFillNa

try:
    p, err = testFillNa(df)
    print(err)
except Exception as e:
    print("Error!\n" + str(e))
    


Correct!


Check if we successfully converted all missing values to the mean value. Display the sum of missing values for the `age` column. 

In [22]:
np.sum(df['age'].isnull(), axis = 0)

0

In the code cell below, do the same for the `hours-per-week` column. Save the result to variable `sum_hours`.

### Graded Cell

The cell below will be graded. Remove the line "raise NotImplementedError()" before writing your code.

In [31]:
sum_hours = np.sum(df['hours-per-week'].isnull(), axis = 0)



### Self-Check

Run the cell below to test the correctness of your code above before submitting for grading. Do not add code or delete code in the cell.

In [32]:
# Run this self-test cell to check your code; 
# do not add code or delete code in this cell
from jn import testSumHours

try:
    p, err = testSumHours(df, sum_hours)
    print(err)
except Exception as e:
    print("Error!\n" + str(e))
    


Correct!
